Working with XLS
==

The incoming raw data is a ".xls" file. It might be an old excel format, though the microsoft excel application does not recognize it as such.

It seems to be html, and can be read as a table by looking at \<tr> elements only and discarding all other tags

In [2]:
import bs4

path = "Data/Raw/2020-01-01 - 2020-01-31/NRA_000000000997.xls"
root = bs4.BeautifulSoup(open(path).read())

First, some code to travel the html tree and print each element on a new line.

The tree being travelled is actually the BeautifulSoup tree, which mostly matches the html tree. Strings are considered elements in their own right in this tree.

The result is the html document, which contains lots of tags and information that are not needed.

In [3]:
show_whitespace = True

# Define elements to ignore entirely, including children
ignore = ["Doctype", "Comment"]

# Global indent variable for printing.
indent = 0


# Print lines indented by the global indent variable
def print_indented(text, *args, **kwargs):

    # Don't bother printing empty lines unless show_whitespace
    # is True. If it is True, mark those lines explicitly as empty.
    text = text.strip()
    if text == "":
        if show_whitespace:
            text = "[EMPTY]"
        else:
            return

    text = " "*indent + text
    print(text, *args, **kwargs)

    
# Print a BeautifulSoup tree element and its children
def print_element(element):
    
    name = type(element).__name__
    
    if name in ignore:
        return
    
    # Check if there is a printer function for that specific 
    # element type
    printer = globals().get("print_" + name)
    
    if printer is not None:
        printer(element)
        
    else:
        # If not, just print the type name
        print_indented(f"[{name}]")

        
# Printer function for html tags
def print_Tag(element):

    global indent
    
    # Opening tag
    print_indented(f"<{element.name}>")
    
    indent += 4
    
    # Print subcontent indented a little
    for child in element.children:
        print_element(child)
    
    indent -= 4 
    
    # Closing tag
    print_indented(f"</{element.name}>") 


# Printer function for string tree elements
def print_NavigableString(element):
    
    # Just print the string contents
    print_indented(element.string)


print_element(root.html)  

<html>
    [EMPTY]
    <head>
        [EMPTY]
        <title>
            Multi-Day Volume Report NRA 000000000997 2020-01-01 to 2020-01-31 Report NRA 000000000997 2020-01-01 to 2020-01-31
        </title>
        [EMPTY]
        <meta>
        </meta>
        [EMPTY]
        <style>
            [Stylesheet]
        </style>
        [EMPTY]
    </head>
    [EMPTY]
    <body>
        [EMPTY]
        <div>
            [EMPTY]
            <b>
                Multi-Day Volume Report NRA 000000000997 2020-01-01 to 2020-01-31
            </b>
            [EMPTY]
            <div>
                [EMPTY]
                <div>
                    [EMPTY]
                    <div>
                        [EMPTY]
                        <div>
                            [EMPTY]
                            <table>
                                [EMPTY]
                                <tr>
                                    <td>
                                        Site Name
                 

                                    </th>
                                    [EMPTY]
                                    <th>
                                        2020-01-20
                                    </th>
                                    [EMPTY]
                                    <th>
                                        2020-01-21
                                    </th>
                                    [EMPTY]
                                    <th>
                                        2020-01-22
                                    </th>
                                    [EMPTY]
                                    <th>
                                        2020-01-23
                                    </th>
                                    [EMPTY]
                                    <th>
                                        2020-01-24
                                    </th>
                                    [EMPTY]
                        

                                        2020-01-28
                                    </th>
                                    [EMPTY]
                                    <th>
                                        2020-01-29
                                    </th>
                                    [EMPTY]
                                    <th>
                                        2020-01-30
                                    </th>
                                    [EMPTY]
                                    <th>
                                        2020-01-31
                                    </th>
                                    [EMPTY]
                                    <th>
                                        Workday
                                    </th>
                                    [EMPTY]
                                    <th>
                                        7 Day
                                    </th>
                         

Take 2: Same approach, but this time ignore most html elements.

For some elements, like \<script> and \<style>, we also want to ignore their contents.

For others, like \<table> and \<th>, we want to ignore them but not their contents

In [4]:
# Define elements to ignore, but keep their children
pass_through = ["div", "b", "td", "th", "body", "ul", "html", "p", "table"]

# Define elements to ignore entirely, including children
ignore = ["Doctype", "Comment", "style", "script", "head", "br"]

# Define elements to retain
keep = ["Tag", "NavigableString", "tr"]

# If an element appears that is not in any list, consider it unexpected, and raise an error.

# global indent level for printing
indent = 0


# Print lines indented by the global indent variable
def print_indented(text, *args, **kwargs):
    text = " "*indent + text
    print(text, *args, **kwargs)


# Print a BeautifulSoup tree element and its children
def print_element(element):
    
    name = type(element).__name__
    
    if name in ignore:
        return
    
    if name not in keep:
        raise TypeError("Unrecognized element type:", name)
    
    
    # At this point, assume there is a printer 
    # function for the element. An error will be raised if there
    # is not.
    printer = globals()["print_" + name]
    printer(element)

        
# Printer function for html tags
def print_Tag(element):

    if element.name in ignore:
        return
    
    elif element.name in pass_through:
        
        # We're ignoring this element, but not its children.
        # Don't print tags, and don't indent children
        
        for child in element.children:
            print_element(child)
    
    elif element.name in keep:

        global indent
        
        print_indented(f"<{element.name}>")
        
        indent += 4
        
        for child in element.children:
            print_element(child)
        
        indent -= 4
        
        print_indented(f"</{element.name}>")
        
    else:
        raise ValueError("Unrecognized tag name:", element.name)
        
        
# Printer function for string tree elements
def print_NavigableString(element):
    
    if element.string.strip() == "":
        return
    
    print_indented(element.string)


print_element(root.html)  

Multi-Day Volume Report NRA 000000000997 2020-01-01 to 2020-01-31
<tr>
    Site Name
    NRA - test site
</tr>
<tr>
    Site ID
    000000000997
</tr>
<tr>
    Grid
    310350240006
</tr>
<tr>
    Description
    Test site for AFMS
</tr>
<tr>
    Setup
    Test
</tr>
<tr>
    Channel
    Each Direction

</tr>
<tr>
    Time Period
    24 hours

</tr>
<tr>
    Class
    Any
</tr>
<tr>
    Exclude data:
    None
</tr>
<tr>
    All directions
</tr>
<tr>
    Wed
    Thu
    Fri
    Sat
    Sun
    Mon
    Tue
    Wed
    Thu
    Fri
    Sat
    Sun
    Mon
    Tue
    Wed
    Thu
    Fri
    Sat
    Sun
    Mon
    Tue
    Wed
    Thu
    Fri
    Sat
    Sun
    Mon
    Tue
    Wed
    Thu
    Fri
    Average
    Total
</tr>
<tr>
    2020-01-01
    2020-01-02
    2020-01-03
    2020-01-04
    2020-01-05
    2020-01-06
    2020-01-07
    2020-01-08
    2020-01-09
    2020-01-10
    2020-01-11
    2020-01-12
    2020-01-13
    2020-01-14
    2020-01-15
    2020-01-16
    2020-01-17
    2020-0

The text result of the above (should) be very simple.

There are only two types of elements - text nodes, which are always leaves, and \<tr> nodes, which can only contain text nodes. This means the tree is only two levels deep.

Top level strings are treated as single cell rows. \<tr> nodes are taken as rows, each child text node is a cell in that row.

In [7]:
# Define elements to ignore, but keep their children
pass_through = ["div", "b", "td", "th", "body", "ul", "html", "p", "table"]

# Define elements to ignore entirely, including children
ignore = ["Doctype", "Comment", "style", "script", "head", "br"]


# Create a table (in the form of a list of lists) from a 
# BeautifulSoup root element
def to_table(element):

    result = list()   

    add_element(result, element)
    
    return result
    

# Add the contents of a BeautifulSoup element to the table.
def add_element(table, element, row_started = False):

    name = type(element).__name__

    if name in ignore:
        return

    elif name == "BeautifulSoup":
        add_element(table, element.html) 
    
    elif name == "Tag":
        add_tag(table, element, row_started)

    elif name == "NavigableString":
        add_string(table, element, row_started)

    else:
        raise TypeError(table, "Unrecognized element type:", name)


# Add the contents of a html tag to the table
def add_tag(table, element, row_started):

    if element.name in ignore:
        # Ignore this element and its children
        return

    if element.name in pass_through:
        # Ignore this element, but add its children
        for child in element.children:
            add_element(table, child, row_started)

        return

    if element.name != "tr":
        # The only element we're interested in giving special
        # meaning to is the <tr> element
        raise ValueError("Unrecognized tag name:", element.name)


    if row_started:
        raise ValueError("There seem to be nested <tr> elements in the tree")

    # Start a new row
    table.append(list())
    row_started = True

    # Add children
    for child in element.children:
        add_element(table, child, row_started)

    # End the row
    row_started = False
    

# Add the contents of a BeautifulSoup string tree element to
# a table.
def add_string(table, element, row_started):

    text = element.string.strip()

    if text == "":
        return

    if row_started:
        table[len(table) - 1].append(text)

    else:
        # If no row is in progress, this is a string on its own
        # Treat it as a single-cell row
        table.append([text])


In [10]:
for i, row in enumerate(to_table(root)):
    
    for element in row:
        print(f"[{element}]", end="")
        
    print()

[Multi-Day Volume Report NRA 000000000997 2020-01-01 to 2020-01-31]
[Site Name][NRA - test site]
[Site ID][000000000997]
[Grid][310350240006]
[Description][Test site for AFMS]
[Setup][Test]
[Channel][Each Direction]
[Time Period][24 hours]
[Class][Any]
[Exclude data:][None]
[All directions]
[Wed][Thu][Fri][Sat][Sun][Mon][Tue][Wed][Thu][Fri][Sat][Sun][Mon][Tue][Wed][Thu][Fri][Sat][Sun][Mon][Tue][Wed][Thu][Fri][Sat][Sun][Mon][Tue][Wed][Thu][Fri][Average][Total]
[2020-01-01][2020-01-02][2020-01-03][2020-01-04][2020-01-05][2020-01-06][2020-01-07][2020-01-08][2020-01-09][2020-01-10][2020-01-11][2020-01-12][2020-01-13][2020-01-14][2020-01-15][2020-01-16][2020-01-17][2020-01-18][2020-01-19][2020-01-20][2020-01-21][2020-01-22][2020-01-23][2020-01-24][2020-01-25][2020-01-26][2020-01-27][2020-01-28][2020-01-29][2020-01-30][2020-01-31][Workday][7 Day][Count]
[00:00:00][48037][48029][48053][48079][48091][48054][48040][48026][48055][48080][48041][48060][48071][48049][48060][48053][48057][48031][480

That's it. The to_table function above is used in the "Data" notebook to read xls files.